In [88]:
from config import client_id, api_key

import pandas as pd
import requests
import json
import time

# Yelp Pet Stores and Pet Services in NYC (ETL): Extract

In [92]:
#function to parse API response for a single Yelp business
def parse_response(response):
    response_json = response.json()
    businesses = []
    
    for business in response_json['businesses']:
        business_json = {}
        categories = []
        
        for category in business['categories']:
            categories.append(category['title'])
        
        business_json['categories'] = categories
        business_json['id'] = business['id']
        business_json['name'] = business['name']
        business_json['is_closed'] = business['is_closed']
        business_json['review_count'] = business['review_count']
        business_json['rating'] = business['rating']
        business_json['zip_code'] = business['location']['zip_code']
        
        businesses.append(business_json)
        
    return businesses
    
#function to call Yelp Fusion API with given search term, location and max search results
def call_yelp(term, location, search_max):
    search_results = []
    search_limit = 50
    search_calls = int(search_max / search_limit)
    
    for i in range(search_calls):
        search_offset = i * search_limit
        url = 'https://api.yelp.com/v3/businesses/search'
        headers = {'Authorization': 'Bearer {}'.format(api_key),}
        url_params = {'term': term.replace(' ', '+'),
                      'location': location.replace(' ', '+'),
                      'limit': search_limit,
                      'offset': search_offset
                     }
        
        response = requests.get(url, headers=headers, params=url_params)
        print('term: {}, offset: {}, response: {}'.format(term, search_offset, response))
        
        search_results.extend(parse_response(response))
        time.sleep(2)
        
    return search_results

In [93]:
#pull and save 1,000 (max) pet stores in NYC
pet_stores = 'Pet Stores'
location = 'New York, NY'
search_max = 1000

yelp_pet_stores = call_yelp(pet_stores, location, search_max)

term: Pet Stores, offset: 0, response: <Response [200]>
term: Pet Stores, offset: 50, response: <Response [200]>
term: Pet Stores, offset: 100, response: <Response [200]>
term: Pet Stores, offset: 150, response: <Response [200]>
term: Pet Stores, offset: 200, response: <Response [200]>
term: Pet Stores, offset: 250, response: <Response [200]>
term: Pet Stores, offset: 300, response: <Response [200]>
term: Pet Stores, offset: 350, response: <Response [200]>
term: Pet Stores, offset: 400, response: <Response [200]>
term: Pet Stores, offset: 450, response: <Response [200]>
term: Pet Stores, offset: 500, response: <Response [200]>
term: Pet Stores, offset: 550, response: <Response [200]>
term: Pet Stores, offset: 600, response: <Response [200]>
term: Pet Stores, offset: 650, response: <Response [200]>
term: Pet Stores, offset: 700, response: <Response [200]>
term: Pet Stores, offset: 750, response: <Response [200]>
term: Pet Stores, offset: 800, response: <Response [200]>
term: Pet Stores,

In [ ]:
#pull and save 1,000 (max) pet services in NYC
pet_services = 'Pet Services'
location = 'New York, NY'
search_max = 1000

yelp_pet_services = call_yelp(pet_services, location, search_max)

# Yelp Pet Stores and Pet Services in NYC (ETL): Transform

In [105]:
stores_and_services = pd.DataFrame.from_dict([yelp_pet_stores, yelp_pet_stores])
stores_and_services = stores_and_services.set_index('id')
stores_and_services

,categories,id,is_closed,name,rating,review_count,zip_code
0,[Pet Stores],gxMLTap163ma5RJNK0EiNg,False,Hoboken Pet,4.5,30,07030
1,[Pet Stores],eTqmm-qOh4nStAcR6Ezn7A,False,Zee.Dog - SoHo,4.5,14,10013
2,[Pet Stores],YtJXZdh3lnuH2JuDtXBqqQ,False,NYC Pet,2.5,15,11211
3,"[Pet Stores, Pet Groomers]",tYZEFqmbGEA0cN6SGz7Swg,False,Petropolis,4.5,76,10006
4,[Pet Stores],A5isQeSW76pMwnXwIpeZEg,False,NYC Pet,4.0,29,11205
5,[Pet Stores],IMw1Jr7T96is-7lccQUT3w,False,Petopia,4.0,52,10009
6,[Pet Stores],ETai16dKCdwZhA5GD2Nbqg,False,Court Street Pet Food,4.0,5,11231
7,[Pet Stores],K0iR_a2f5ISP8NY663-P3A,False,Fussy Friends Pet Supply,4.5,59,07302
8,[Pet Stores],mgmejPGqdZ2KNiaGZTUlRA,False,The Pet Bar,4.5,28,10012
9,[Pet Stores],t9_yKbLU-aVCD7c8tDhclw,False,Nyc Pet Knickerbocker,3.5,18,11237


# Yelp Pet Stores and Pet Services in NYC (ETL): Load

In [ ]:
with open('test.json') as f:
    data = json.load(f)

with open('test.json', 'w') as f:
    json.dump(data, f)